# 🎯 Integration & Comparison: Phase 1 Classical ML

**Цель:** Финальное сравнение всех изученных методов на едином датасете

---

## 🎯 Цели ноутбука

1. **Объединение знаний** из всех ноутбуков Phase 1
2. **Честное сравнение** всех методов на одном датасете
3. **Практические рекомендации** по выбору алгоритмов
4. **Best practices** для production ML систем

---

## 📚 Что мы изучили в Phase 1:

| Ноутбук | Тема | Ключевые концепции |
|---------|------|-------------------|
| **01** | XGBoost Deep Dive | Gradient boosting, second-order optimization, regularization |
| **02** | LightGBM Deep Dive | Histogram-based, GOSS, EFB, leaf-wise growth |
| **03** | CatBoost Deep Dive | Ordered boosting, categorical features, symmetric trees |
| **04** | Stacking Ensemble | Meta-learning, out-of-fold predictions, diversity |
| **05** | Imbalanced Data | SMOTE, cost-sensitive learning, threshold optimization |
| **06** | Advanced Feature Engineering | Polynomial, target encoding, feature selection |

---

## 💼 Бизнес-задача: Telco Customer Churn

**Контекст:** Телеком компания теряет 27% клиентов ежегодно.

**Проблемы:**
- 📊 **Imbalanced data:** ~27% churn (не extreme, но moderate)
- 🔢 **Категориальные признаки:** Услуги, тип контракта (high cardinality)
- 💰 **Разная стоимость ошибок:** FN (упущенный churn) дороже FP
- 🎯 **Бизнес-цель:** Предотвратить churn с retention campaigns

**Метрики:**
- **ROC-AUC:** Общая способность различать классы
- **Precision/Recall/F1:** Баланс FP/FN
- **Business Cost:** Weighted cost (FN = $500, FP = $50)

**Почему этот датасет?**
- ✅ Позволяет применить **все** техники Phase 1
- ✅ Категориальные признаки → тестируем CatBoost, Target Encoding
- ✅ Moderate imbalance → SMOTE, class weights
- ✅ Малый размер (~7k) → можно быстро экспериментировать

---

## 📚 Часть 1: Теоретическое сравнение методов

### 1.1 Gradient Boosting семейство

#### Общая идея Gradient Boosting

**Формула (Friedman, 2001):**

$$F_m(x) = F_{m-1}(x) + \nu \cdot h_m(x)$$

где:
- $F_m(x)$ — ансамбль из $m$ деревьев
- $h_m(x)$ — новое дерево, обучаемое на градиентах
- $\nu$ — learning rate (shrinkage)

**Градиент:**

$$g_i = \frac{\partial L(y_i, F_{m-1}(x_i))}{\partial F_{m-1}(x_i)}$$

Каждое дерево приближает **anti-gradient** loss функции.

---

### 1.2 XGBoost vs LightGBM vs CatBoost

#### Детальное сравнение

**1. XGBoost (Chen & Guestrin, 2016)**

**Ключевая идея:** Second-order optimization

$$L = \sum_{i=1}^n \left[ g_i f_t(x_i) + \frac{1}{2} h_i f_t^2(x_i) \right] + \Omega(f_t)$$

где:
- $g_i$ — first-order gradient
- $h_i$ — second-order gradient (Hessian)
- $\Omega(f_t) = \gamma T + \frac{1}{2}\lambda \sum_{j=1}^T w_j^2$ — regularization

**Преимущества:**
- ✅ Точная оптимизация (Newton's method)
- ✅ Сильная регуляризация (L1/L2, gamma, alpha)
- ✅ Стабильность и зрелость
- ✅ Отличная документация и community

**Недостатки:**
- ❌ Медленнее LightGBM на больших данных
- ❌ Больше памяти (pre-sorted или histogram)
- ❌ Категориальные нужно кодировать вручную

---

**2. LightGBM (Ke et al., 2017)**

**Ключевые идеи:**

**a) Histogram-based splits**

Вместо сортировки всех значений → bins (обычно 255):

$$\text{bin}(x) = \left\lfloor \frac{x - \min(x)}{\text{bin\_width}} \right\rfloor$$

**b) GOSS (Gradient-based One-Side Sampling)**

```
1. Сортировать примеры по |gradient|
2. Взять top-a% (большие градиенты)
3. Случайно взять b% из оставшихся
4. Взвесить малые градиенты на (1-a)/b
```

**c) Leaf-wise (best-first) growth**

Вместо level-wise → выбираем leaf с максимальным gain:

$$\text{Gain} = \frac{1}{2} \left[ \frac{G_L^2}{H_L + \lambda} + \frac{G_R^2}{H_R + \lambda} - \frac{(G_L + G_R)^2}{H_L + H_R + \lambda} \right] - \gamma$$

**Преимущества:**
- ✅ **5-20x быстрее** XGBoost на больших данных
- ✅ Меньше памяти (~50%)
- ✅ Native categorical support
- ✅ Лучше на sparse данных (EFB)

**Недостатки:**
- ❌ Склонен к overfitting (leaf-wise aggressive)
- ❌ Требует тщательного tuning
- ❌ Хуже на малых данных (<10k)

---

**3. CatBoost (Prokhorenkova et al., 2018)**

**Ключевые идеи:**

**a) Ordered Boosting**

Решение проблемы **prediction shift** — используем разные модели:

$$F_{<i}(x_i) = \sum_{t=1}^{T} h_t^{<i}(x_i)$$

Модель $F_{<i}$ обучена **без** примера $i$.

**b) Ordered Target Statistics (для categorical)**

$$\hat{x}_k^i = \frac{\sum_{j=1}^{p-1} [x_j = x_k] \cdot y_{\sigma_j} + a \cdot p}{\sum_{j=1}^{p-1} [x_j = x_k] + a}$$

где $\sigma$ — случайная перестановка, $p$ — позиция примера $i$, $a$ — prior weight.

**c) Symmetric (Oblivious) Trees**

Все листья на одном уровне используют одинаковый split → быстрый inference.

**Преимущества:**
- ✅ **Лучшие categorical features** из коробки
- ✅ Меньше overfitting (ordered boosting)
- ✅ Отличное качество **по умолчанию** (меньше tuning)
- ✅ GPU support

**Недостатки:**
- ❌ Медленнее обучение (ordered boosting сложнее)
- ❌ Больше памяти при обучении
- ❌ Symmetric trees могут быть менее гибкими

---

### 1.3 Когда использовать какой алгоритм?

#### Практическая таблица выбора

| Сценарий | Рекомендация | Почему |
|----------|-------------|--------|
| **Малые данные (<10k)** | XGBoost | Стабильнее, level-wise безопаснее |
| **Средние данные (10k-1M)** | XGBoost или CatBoost | Зависит от categorical features |
| **Большие данные (>1M)** | LightGBM | В 5-20x быстрее |
| **Много categorical** | CatBoost | Native support, ordered target stats |
| **Разреженные данные** | LightGBM | EFB оптимизация |
| **Первая модель (baseline)** | XGBoost | Меньше tuning, хорошие defaults |
| **Production-critical** | XGBoost | Зрелость, стабильность |
| **Скорость inference** | CatBoost | Symmetric trees быстрее |
| **Ограниченная память** | LightGBM | Histogram-based |
| **Kaggle competition** | Ensemble всех трех! | Diversity → лучше ensemble |

#### Эмпирические результаты

**Из Kaggle competitions и research:**

1. **Качество (ROC-AUC):**
   - Большие данные: LightGBM ≥ CatBoost > XGBoost (разница ~0.5-2%)
   - Малые данные: XGBoost ≥ CatBoost ≥ LightGBM
   - С категориальными: CatBoost часто лучший

2. **Скорость обучения:**
   - LightGBM: **5-20x** быстрее XGBoost на >100k примеров
   - CatBoost: **2-3x** медленнее XGBoost (ordered boosting)
   - XGBoost: baseline скорость

3. **Память:**
   - LightGBM: **~50%** от XGBoost
   - CatBoost: **~150%** от XGBoost при обучении
   - XGBoost: baseline

4. **Tuning сложность:**
   - XGBoost: Средняя (хорошие defaults)
   - LightGBM: Высокая (leaf-wise нужен контроль)
   - CatBoost: **Низкая** (отличные defaults!)

---

### 1.4 Stacking: Когда и как?

#### Теория Stacking

**Wolpert's Stacked Generalization (1992):**

**Level 0 (Base learners):**

$$\hat{y}_i^{(k)} = f_k(x_i), \quad k = 1, \ldots, K$$

**Level 1 (Meta-learner):**

$$\hat{y}_i = g\left( \hat{y}_i^{(1)}, \hat{y}_i^{(2)}, \ldots, \hat{y}_i^{(K)} \right)$$

**Out-of-fold predictions (избегаем overfitting):**

```
For fold k in K-Fold:
  1. Train base learners на train_k
  2. Predict на val_k → oof_predictions_k
  3. Predict на test → test_predictions_k
  
Meta features = concat(oof_predictions_1, ..., oof_predictions_K)
Meta learner.fit(Meta features, y)
```

#### Когда Stacking работает?

**Ключевое условие:** Diversity базовых моделей!

**Ambiguity Decomposition (Krogh & Vedelsby, 1995):**

$$E_{\text{ensemble}} = \bar{E} - \bar{A}$$

где:
- $\bar{E}$ — средняя ошибка базовых моделей
- $\bar{A}$ — средняя ambiguity (diversity)

**Чем больше diversity, тем меньше ошибка ensemble!**

#### Как создать diversity?

1. **Разные алгоритмы:** XGBoost + LightGBM + CatBoost + Logistic Regression
2. **Разные признаки:** Один с polynomial, другой с target encoding
3. **Разные гиперпараметры:** Shallow vs deep trees
4. **Разные подвыборки:** Bagging на разных частях данных

#### Практические рекомендации:

✅ **Используйте Stacking если:**
- Есть время на обучение нескольких моделей
- Нужны последние проценты качества (Kaggle, research)
- Базовые модели достаточно разные

❌ **НЕ используйте Stacking если:**
- Нужна скорость inference (в 3-5x медленнее)
- Ограниченные ресурсы (обучение всех моделей долго)
- Базовые модели очень похожи (мало diversity)
- Production система с простотой поддержки

**Типичный прирост:** 0.5-3% ROC-AUC поверх лучшей базовой модели.

---

### 1.5 Feature Engineering: Возврат инвестиций

#### Влияние на качество модели

**Эмпирическое правило (Andrew Ng):**

| Компонент | Влияние на качество | Время инвестиций |
|-----------|---------------------|------------------|
| **Feature Engineering** | 60-70% | Высокое |
| Algorithm selection | 15-20% | Среднее |
| Hyperparameter tuning | 10-15% | Среднее |
| Ensemble | 5-10% | Низкое |

#### Эффективность техник

**Из нашего опыта (Notebook 06):**

| Техника | Прирост качества | Когда использовать |
|---------|-----------------|-------------------|
| **Log transform** | 5-10% | Skewed данные (цены, площади) |
| **Target encoding** | 3-7% | High cardinality categorical |
| **Polynomial features** | 5-15% | Линейные модели, малые данные |
| **Feature selection** | 2-5% | Много шума, overfitting |
| **Domain features** | 10-30%! | Когда есть domain knowledge |

#### Важность для разных алгоритмов

**Linear models (Logistic Regression, SVM):**
- ⭐⭐⭐⭐⭐ Критично! (нужны interactions, scaling, transformations)

**Tree-based models (XGBoost, RF):**
- ⭐⭐⭐ Полезно (domain features, target encoding)
- Polynomial interactions НЕ нужны (деревья сами их находят)

**Neural Networks:**
- ⭐⭐ Может помочь, но сети часто учат сами (feature learning)

---

### 1.6 Imbalanced Data: Стратегия выбора техники

#### Классификация по степени imbalance

| Imbalance ratio | Категория | Рекомендация |
|----------------|-----------|-------------|
| 1:1 - 1:4 | Balanced/Mild | Ничего не нужно |
| 1:4 - 1:20 | Moderate | Class weights |
| 1:20 - 1:100 | High | SMOTE + class weights |
| 1:100+ | Extreme | Advanced (Focal loss, sampling) |

#### Техники и их применимость

**1. Class Weights (scale_pos_weight в XGBoost)**

$$w_{\text{pos}} = \frac{N_{\text{neg}}}{N_{\text{pos}}}$$

- ✅ Простота
- ✅ Работает для tree-based models
- ✅ Нет изменения размера данных
- ❌ Не создает новые примеры

**2. SMOTE (Synthetic Minority Over-sampling)**

$$x_{\text{new}} = x_i + \lambda \cdot (x_{\text{nn}} - x_i), \quad \lambda \in [0, 1]$$

- ✅ Создает новые примеры minority класса
- ✅ Помогает с diversity
- ❌ Может создать noise
- ❌ Увеличивает размер данных

**3. Hybrid: SMOTE + Class Weights**

- ✅ Лучшее из обоих миров
- Moderate SMOTE (до 1:10) + weights

**4. Focal Loss (Lin et al., 2017)**

$$FL(p_t) = -\alpha_t (1 - p_t)^\gamma \log(p_t)$$

- ✅ Фокусируется на hard examples
- ✅ Автоматически down-weights easy examples
- ❌ Нужна custom реализация

#### Метрики для imbalanced data

**НЕ используйте Accuracy!**

**Используйте:**
1. **Precision/Recall/F1**
2. **PR-AUC** (лучше ROC-AUC для imbalanced)
3. **Business cost** (weighted FP и FN)

---

## Теоретическая часть завершена! Переходим к практике 🚀

---

## 📊 Часть 2: Практическое сравнение

### 2.1 Импорт библиотек

In [ ]:
# Основные библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
import time
warnings.filterwarnings('ignore')

# Gradient Boosting
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
import catboost as cb
from catboost import CatBoostClassifier

# Baseline models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, cross_val_predict

# Feature engineering
from sklearn.preprocessing import PolynomialFeatures, PowerTransformer
from sklearn.feature_selection import SelectKBest, f_classif

# Imbalanced data
try:
    from imblearn.over_sampling import SMOTE
    HAS_IMBLEARN = True
except ImportError:
    print('⚠️ imbalanced-learn не установлен')
    HAS_IMBLEARN = False

# Metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score,
    confusion_matrix, classification_report,
    roc_curve, precision_recall_curve
)

# Настройка визуализации
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Seed
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print('✅ Библиотеки загружены')
print(f'XGBoost: {xgb.__version__}')
print(f'LightGBM: {lgb.__version__}')
print(f'CatBoost: {cb.__version__}')

### 2.2 Загрузка и подготовка данных

In [ ]:
# Загрузка Telco Churn данных
import os

data_path = '../../data/telco_churn.csv'

if not os.path.exists(data_path):
    print('❌ Файл не найден!')
else:
    df = pd.read_csv(data_path)
    print(f'✅ Данные загружены: {df.shape[0]:,} строк, {df.shape[1]} столбцов')
    print(f'Target: Churn')

In [ ]:
# Первый взгляд
print('Первые строки:')
df.head()

In [ ]:
# EDA: Class balance
churn_counts = df['Churn'].value_counts()
churn_pct = df['Churn'].value_counts(normalize=True) * 100

print('Class distribution:')
print(f'No: {churn_counts["No"]} ({churn_pct["No"]:.1f}%)')
print(f'Yes: {churn_counts["Yes"]} ({churn_pct["Yes"]:.1f}%)')
print(f'\nImbalance ratio: 1:{churn_counts["No"] / churn_counts["Yes"]:.1f}')

# Визуализация
plt.figure(figsize=(8, 5))
churn_counts.plot(kind='bar', color=['green', 'red'], alpha=0.7, edgecolor='black')
plt.title('Class Distribution: Churn')
plt.xlabel('Churn')
plt.ylabel('Count')
plt.xticks(rotation=0)
for i, (idx, val) in enumerate(churn_counts.items()):
    plt.text(i, val + 100, f'{val} ({churn_pct[idx]:.1f}%)', ha='center')
plt.tight_layout()
plt.show()

print('\n🔍 Moderate imbalance (~27% churn) → Class weights или moderate SMOTE')

In [ ]:
# Подготовка данных
# Удаляем customerID (не нужен)
if 'customerID' in df.columns:
    df = df.drop('customerID', axis=1)

# TotalCharges иногда имеет ' ' вместо чисел - исправляем
if 'TotalCharges' in df.columns:
    df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
    df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)

# Разделяем на numeric и categorical
numeric_features = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = df.select_dtypes(include=['object']).columns.tolist()
categorical_features.remove('Churn')  # Target

print(f'Numeric features ({len(numeric_features)}): {numeric_features}')
print(f'Categorical features ({len(categorical_features)}): {categorical_features[:5]}...')

In [ ]:
# Target encoding
y = (df['Churn'] == 'Yes').astype(int)

# Features
X = df.drop('Churn', axis=1)

print(f'X shape: {X.shape}')
print(f'y shape: {y.shape}')
print(f'y distribution: {y.value_counts().to_dict()}')

In [ ]:
# Train/test split (stratified!)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

print(f'Train: {X_train.shape[0]:,} samples')
print(f'Test: {X_test.shape[0]:,} samples')
print(f'Train churn rate: {y_train.mean():.1%}')
print(f'Test churn rate: {y_test.mean():.1%}')

### 2.3 Подготовка разных версий данных\n",

Создадим несколько версий для разных моделей:
1. **One-hot encoded** — для Logistic Regression, XGBoost, LightGBM
2. **Native categorical** — для CatBoost
3. **Target encoded** — альтернативный вариант

In [ ]:
# Version 1: One-hot encoding
X_train_ohe = pd.get_dummies(X_train, columns=categorical_features, drop_first=True)
X_test_ohe = pd.get_dummies(X_test, columns=categorical_features, drop_first=True)

# Выровнять колонки
X_train_ohe, X_test_ohe = X_train_ohe.align(X_test_ohe, join='left', axis=1, fill_value=0)

print(f'One-hot encoded: {X_train_ohe.shape[1]} features')

In [ ]:
# Version 2: Label encoding для CatBoost (он поймет categorical сам)
X_train_cat = X_train.copy()
X_test_cat = X_test.copy()

# CatBoost принимает categorical как индексы категорий
cat_indices = []
for i, col in enumerate(X_train_cat.columns):
    if col in categorical_features:
        le = LabelEncoder()
        X_train_cat[col] = le.fit_transform(X_train_cat[col].astype(str))
        X_test_cat[col] = le.transform(X_test_cat[col].astype(str))
        cat_indices.append(i)

print(f'CatBoost version: {len(cat_indices)} categorical features at indices {cat_indices[:5]}...')

In [ ]:
# Масштабирование для Logistic Regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_ohe)
X_test_scaled = scaler.transform(X_test_ohe)

print('✅ Все версии данных подготовлены')

### 2.4 Функция оценки моделей

In [ ]:
def evaluate_model(model, X_tr, X_te, y_tr, y_te, name, train_time=None):
    """
    Комплексная оценка модели
    """
    # Предсказания
    y_pred = model.predict(X_te)
    y_proba = model.predict_proba(X_te)[:, 1]
    
    # Метрики
    acc = accuracy_score(y_te, y_pred)
    prec = precision_score(y_te, y_pred)
    rec = recall_score(y_te, y_pred)
    f1 = f1_score(y_te, y_pred)
    roc_auc = roc_auc_score(y_te, y_proba)
    pr_auc = average_precision_score(y_te, y_proba)
    
    # Business cost (FN = $500, FP = $50)
    tn, fp, fn, tp = confusion_matrix(y_te, y_pred).ravel()
    cost = fn * 500 + fp * 50
    
    # Вывод
    print(f'\n📊 {name}:')
    print(f'  Accuracy: {acc:.4f}')
    print(f'  Precision: {prec:.4f}')
    print(f'  Recall: {rec:.4f}')
    print(f'  F1-score: {f1:.4f}')
    print(f'  ROC-AUC: {roc_auc:.4f}')
    print(f'  PR-AUC: {pr_auc:.4f}')
    print(f'  Business Cost: ${cost:,}')
    if train_time:
        print(f'  Train Time: {train_time:.2f}s')
    print(f'  Confusion Matrix: TN={tn}, FP={fp}, FN={fn}, TP={tp}')
    
    return {
        'Model': name,
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1': f1,
        'ROC-AUC': roc_auc,
        'PR-AUC': pr_auc,
        'Cost': cost,
        'Time': train_time if train_time else 0,
        'TP': tp,
        'FP': fp,
        'TN': tn,
        'FN': fn
    }

# Инициализируем результаты
results = []

print('✅ Функция оценки создана')

### 2.5 Baseline: Logistic Regression

In [ ]:
# Logistic Regression baseline
start = time.time()
lr = LogisticRegression(max_iter=1000, random_state=RANDOM_STATE)
lr.fit(X_train_scaled, y_train)
lr_time = time.time() - start

lr_results = evaluate_model(lr, X_train_scaled, X_test_scaled, y_train, y_test, 
                            'Logistic Regression', lr_time)
results.append(lr_results)

### 2.6 XGBoost: Baseline и Tuned

In [ ]:
# XGBoost baseline
start = time.time()
xgb_base = XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=RANDOM_STATE,
    verbosity=0
)
xgb_base.fit(X_train_ohe, y_train)
xgb_base_time = time.time() - start

xgb_base_results = evaluate_model(xgb_base, X_train_ohe, X_test_ohe, y_train, y_test,
                                  'XGBoost Baseline', xgb_base_time)
results.append(xgb_base_results)

In [ ]:
# XGBoost tuned с class weights
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

start = time.time()
xgb_tuned = XGBClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=4,
    min_child_weight=3,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=1.0,
    scale_pos_weight=scale_pos_weight,  # Class weights!
    random_state=RANDOM_STATE,
    verbosity=0
)
xgb_tuned.fit(X_train_ohe, y_train)
xgb_tuned_time = time.time() - start

xgb_tuned_results = evaluate_model(xgb_tuned, X_train_ohe, X_test_ohe, y_train, y_test,
                                   'XGBoost Tuned + Weights', xgb_tuned_time)
results.append(xgb_tuned_results)

### 2.7 LightGBM: Baseline и Tuned

In [ ]:
# LightGBM baseline
start = time.time()
lgbm_base = LGBMClassifier(
    n_estimators=100,
    learning_rate=0.1,
    num_leaves=31,
    random_state=RANDOM_STATE,
    verbose=-1
)
lgbm_base.fit(X_train_ohe, y_train)
lgbm_base_time = time.time() - start

lgbm_base_results = evaluate_model(lgbm_base, X_train_ohe, X_test_ohe, y_train, y_test,
                                   'LightGBM Baseline', lgbm_base_time)
results.append(lgbm_base_results)

In [ ]:
# LightGBM tuned с class weights
start = time.time()
lgbm_tuned = LGBMClassifier(
    n_estimators=200,
    learning_rate=0.05,
    num_leaves=31,
    max_depth=6,
    min_child_samples=20,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=1.0,
    scale_pos_weight=scale_pos_weight,
    random_state=RANDOM_STATE,
    verbose=-1
)
lgbm_tuned.fit(X_train_ohe, y_train)
lgbm_tuned_time = time.time() - start

lgbm_tuned_results = evaluate_model(lgbm_tuned, X_train_ohe, X_test_ohe, y_train, y_test,
                                    'LightGBM Tuned + Weights', lgbm_tuned_time)
results.append(lgbm_tuned_results)

### 2.8 CatBoost: Native Categorical Features

In [ ]:
# CatBoost baseline (с native categorical)
start = time.time()
cat_base = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    depth=6,
    cat_features=cat_indices,  # NATIVE CATEGORICAL!
    random_state=RANDOM_STATE,
    verbose=0
)
cat_base.fit(X_train_cat, y_train)
cat_base_time = time.time() - start

cat_base_results = evaluate_model(cat_base, X_train_cat, X_test_cat, y_train, y_test,
                                  'CatBoost Baseline (Native Cat)', cat_base_time)
results.append(cat_base_results)

In [ ]:
# CatBoost tuned с class weights
start = time.time()
cat_tuned = CatBoostClassifier(
    iterations=200,
    learning_rate=0.05,
    depth=6,
    l2_leaf_reg=3,
    cat_features=cat_indices,
    class_weights=[1, scale_pos_weight],  # Class weights!
    random_state=RANDOM_STATE,
    verbose=0
)
cat_tuned.fit(X_train_cat, y_train)
cat_tuned_time = time.time() - start

cat_tuned_results = evaluate_model(cat_tuned, X_train_cat, X_test_cat, y_train, y_test,
                                   'CatBoost Tuned + Weights', cat_tuned_time)
results.append(cat_tuned_results)

### 2.9 Imbalanced Data: SMOTE

In [ ]:
# SMOTE + XGBoost
if HAS_IMBLEARN:
    smote = SMOTE(sampling_strategy=0.7, random_state=RANDOM_STATE)  # До 70% minority
    X_train_smote, y_train_smote = smote.fit_resample(X_train_ohe, y_train)
    
    print(f'После SMOTE: {X_train_smote.shape[0]} samples')
    print(f'Churn rate: {y_train_smote.mean():.1%}')
    
    start = time.time()
    xgb_smote = XGBClassifier(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=4,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=RANDOM_STATE,
        verbosity=0
    )
    xgb_smote.fit(X_train_smote, y_train_smote)
    xgb_smote_time = time.time() - start
    
    xgb_smote_results = evaluate_model(xgb_smote, X_train_smote, X_test_ohe, 
                                       y_train_smote, y_test,
                                       'XGBoost + SMOTE', xgb_smote_time)
    results.append(xgb_smote_results)
else:
    print('⚠️ SMOTE пропущен (imbalanced-learn не установлен)')

### 2.10 Stacking Ensemble

In [ ]:
# Stacking: XGBoost + LightGBM + CatBoost → Logistic Regression
print('🚀 Создаем Stacking Ensemble...')

# Out-of-fold predictions для meta features
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

# Базовые модели (уже обучены)
base_models = [
    ('xgb', xgb_tuned, X_train_ohe, X_test_ohe),
    ('lgbm', lgbm_tuned, X_train_ohe, X_test_ohe),
    ('cat', cat_tuned, X_train_cat, X_test_cat)
]

# OOF predictions
oof_train = np.zeros((X_train.shape[0], len(base_models)))
oof_test = np.zeros((X_test.shape[0], len(base_models)))

for i, (name, model, X_tr, X_te) in enumerate(base_models):
    # Используем уже обученные модели для test
    oof_test[:, i] = model.predict_proba(X_te)[:, 1]
    
    # OOF для train через cross-validation
    oof_preds = cross_val_predict(model, X_tr, y_train, cv=cv, method='predict_proba')[:, 1]
    oof_train[:, i] = oof_preds
    
    print(f'✅ {name}: OOF готов')

print(f'Meta features shape: train {oof_train.shape}, test {oof_test.shape}')

In [ ]:
# Meta-learner: Logistic Regression
start = time.time()
meta_learner = LogisticRegression(random_state=RANDOM_STATE)
meta_learner.fit(oof_train, y_train)
stacking_time = time.time() - start

# Предсказания
stacking_pred_proba = meta_learner.predict_proba(oof_test)[:, 1]
stacking_pred = (stacking_pred_proba > 0.5).astype(int)

# Оценка
acc = accuracy_score(y_test, stacking_pred)
prec = precision_score(y_test, stacking_pred)
rec = recall_score(y_test, stacking_pred)
f1 = f1_score(y_test, stacking_pred)
roc_auc = roc_auc_score(y_test, stacking_pred_proba)
pr_auc = average_precision_score(y_test, stacking_pred_proba)

tn, fp, fn, tp = confusion_matrix(y_test, stacking_pred).ravel()
cost = fn * 500 + fp * 50

print(f'\n📊 Stacking Ensemble:')
print(f'  Accuracy: {acc:.4f}')
print(f'  Precision: {prec:.4f}')
print(f'  Recall: {rec:.4f}')
print(f'  F1-score: {f1:.4f}')
print(f'  ROC-AUC: {roc_auc:.4f}')
print(f'  PR-AUC: {pr_auc:.4f}')
print(f'  Business Cost: ${cost:,}')
print(f'  Meta-learner weights: {meta_learner.coef_[0]}')

stacking_results = {
    'Model': 'Stacking Ensemble',
    'Accuracy': acc,
    'Precision': prec,
    'Recall': rec,
    'F1': f1,
    'ROC-AUC': roc_auc,
    'PR-AUC': pr_auc,
    'Cost': cost,
    'Time': stacking_time,
    'TP': tp,
    'FP': fp,
    'TN': tn,
    'FN': fn
}
results.append(stacking_results)

### 2.11 Финальное сравнение всех методов

In [ ]:
# Создаем сравнительную таблицу
comparison_df = pd.DataFrame(results)
comparison_df = comparison_df.sort_values('ROC-AUC', ascending=False)

print('\n' + '='*100)
print('🏆 ФИНАЛЬНОЕ СРАВНЕНИЕ ВСЕХ МЕТОДОВ PHASE 1')
print('='*100)
print(comparison_df[['Model', 'ROC-AUC', 'F1', 'Precision', 'Recall', 'Cost', 'Time']].to_string(index=False))
print('='*100)

In [ ]:
# Визуализация сравнения
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# ROC-AUC
ax = axes[0, 0]
bars = ax.barh(comparison_df['Model'], comparison_df['ROC-AUC'], color='skyblue', edgecolor='black')
ax.set_xlabel('ROC-AUC')
ax.set_title('ROC-AUC Score')
ax.axvline(comparison_df['ROC-AUC'].max(), color='red', linestyle='--', alpha=0.5)

# F1-score
ax = axes[0, 1]
ax.barh(comparison_df['Model'], comparison_df['F1'], color='lightgreen', edgecolor='black')
ax.set_xlabel('F1-score')
ax.set_title('F1 Score')

# Precision vs Recall
ax = axes[0, 2]
ax.scatter(comparison_df['Recall'], comparison_df['Precision'], s=200, alpha=0.6, edgecolor='black')
for i, model in enumerate(comparison_df['Model']):
    ax.annotate(model.split()[0], 
                (comparison_df['Recall'].iloc[i], comparison_df['Precision'].iloc[i]),
                fontsize=8)
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_title('Precision vs Recall')
ax.grid(True, alpha=0.3)

# Business Cost
ax = axes[1, 0]
comparison_sorted_cost = comparison_df.sort_values('Cost')
ax.barh(comparison_sorted_cost['Model'], comparison_sorted_cost['Cost'], 
        color='lightcoral', edgecolor='black')
ax.set_xlabel('Business Cost ($)')
ax.set_title('Business Cost (Lower is Better)')

# Training Time
ax = axes[1, 1]
comparison_sorted_time = comparison_df.sort_values('Time')
ax.barh(comparison_sorted_time['Model'], comparison_sorted_time['Time'], 
        color='wheat', edgecolor='black')
ax.set_xlabel('Time (seconds)')
ax.set_title('Training Time')

# Confusion Matrix for best model
ax = axes[1, 2]
best_model = comparison_df.iloc[0]
cm = np.array([[best_model['TN'], best_model['FP']], 
               [best_model['FN'], best_model['TP']]])
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax, cbar=False)
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title(f'Best Model: {best_model["Model"]}')
ax.set_xticklabels(['No Churn', 'Churn'])
ax.set_yticklabels(['No Churn', 'Churn'])

plt.tight_layout()
plt.show()

## 🎯 Выводы и Рекомендации

### Что мы выяснили:

#### 1. Качество моделей (ROC-AUC)

Типичные результаты (зависит от данных):
- **Stacking Ensemble:** Обычно лучший (~0.85-0.87)
- **CatBoost Tuned:** Близко к stacking (~0.84-0.86)
- **XGBoost/LightGBM Tuned:** Сопоставимо (~0.83-0.85)
- **Базовые boosting:** Хорошо (~0.81-0.83)
- **Logistic Regression:** Baseline (~0.75-0.78)

**Вывод:** Разница между tuned моделями часто <2% ROC-AUC

#### 2. Скорость обучения

- **LightGBM:** Самый быстрый (особенно на больших данных)
- **XGBoost:** Средняя скорость
- **CatBoost:** Медленнее (ordered boosting сложнее)
- **Stacking:** Самый медленный (обучение всех моделей)

#### 3. Categorical features

**CatBoost с native categorical показывает отличные результаты БЕЗ:**
- One-hot encoding
- Target encoding
- Feature engineering

**Вывод:** Для датасетов с категориальными - CatBoost лучший выбор из коробки!

#### 4. Imbalanced data

**Class weights vs SMOTE:**
- Class weights: Проще, быстрее, работает хорошо для moderate imbalance
- SMOTE: Может помочь для extreme imbalance, но риск noise
- **Hybrid** (moderate SMOTE + weights): Часто лучший вариант

#### 5. Stacking

**Прирост:** Обычно 0.5-2% поверх лучшей базовой модели

**Стоит ли?**
- ✅ Kaggle, research: Да!
- ❌ Production с ограничениями: Вероятно нет

---

### Практические рекомендации для Production

#### Сценарий 1: Быстрый старт

**Рекомендация:** XGBoost с дефолтными параметрами

```python
XGBClassifier(n_estimators=100, learning_rate=0.1)
```

- ✅ Хорошее качество из коробки
- ✅ Стабильность
- ✅ Минимальный tuning

#### Сценарий 2: Много категориальных признаков

**Рекомендация:** CatBoost с native categorical

```python
CatBoostClassifier(iterations=100, cat_features=cat_indices)
```

- ✅ Не нужен one-hot encoding
- ✅ Отличное качество по умолчанию
- ✅ Ordered target statistics автоматически

#### Сценарий 3: Большие данные (>1M строк)

**Рекомендация:** LightGBM

```python
LGBMClassifier(n_estimators=100, num_leaves=31)
```

- ✅ В 5-20x быстрее XGBoost
- ✅ Меньше памяти
- ⚠️ Требует tuning для избежания overfitting

#### Сценарий 4: Максимальное качество (Kaggle)

**Рекомендация:** Stacking Ensemble

```python
Base: XGBoost + LightGBM + CatBoost + RF
Meta: LogisticRegression / XGBoost
```

- ✅ Выжимаем последние проценты
- ✅ Diversity базовых моделей
- ❌ Сложность и время обучения

#### Сценарий 5: Imbalanced data

**Moderate imbalance (1:10):**
```python
scale_pos_weight = n_negative / n_positive
XGBClassifier(scale_pos_weight=scale_pos_weight)
```

**Extreme imbalance (1:100+):**
```python
smote = SMOTE(sampling_strategy=0.1)  # Moderate oversample
+ class_weights
```

---

### Общие Best Practices

1. **Всегда начинайте с baseline** (Logistic Regression)
2. **Используйте cross-validation** для честной оценки
3. **Class imbalance:** Используйте PR-AUC, не accuracy!
4. **Feature engineering:** 60-70% качества, инвестируйте время
5. **Hyperparameter tuning:** Только после feature engineering
6. **Ensemble:** Последний шаг, если нужны проценты
7. **Production:** Простота > сложность (XGBoost или CatBoost)

---

## 🎉 Phase 1 Classical ML завершен!

**Мы изучили:**
- ✅ XGBoost, LightGBM, CatBoost в деталях
- ✅ Stacking Ensemble
- ✅ Imbalanced data техники
- ✅ Advanced Feature Engineering
- ✅ Практическое сравнение на реальных данных

**Следующие шаги:**
- Phase 2: Deep Learning (MLP, CNN, RNN, Transformers)
- Phase 3: Time Series
- Phase 4: NLP
- Phase 5: Computer Vision

**Поздравляю! 🚀**
